### Temporary Notebook to try out a few things

Notation for Ising and QUBO: [Ref.](https://doi.org/10.1103/PhysRevResearch.1.033142)

In [4]:
import sys
import numpy as np
sys.path.append("../tdvp_qa")
from generator import generate_graph

In [5]:

def convert_to_Ising_matrix(N_verts, Jz):
    """
    Convert Jz object to an Ising spin glass matrix
    """
    Jz_matrix = np.zeros((N_verts, N_verts))
    n_edges = Jz.shape[0]
    for e in range(n_edges):
        n1 = int(Jz[e, 0])
        n2 = int(Jz[e, 1])
        Jz_matrix[n1, n2] = Jz[e, 2] # upper triangular
    return Jz_matrix

def Ising_to_QUBO(J, h):
    """
    Implement the mapping: H = 2*E + C, with E = x.T Q x
    H = Ising
    E = QUBO

    Explicitly: energy_Ising==energy_QUBO
    
    energy_Ising = 0.5 * sigma_Ising.T @ J @ sigma_Ising + np.dot(sigma_Ising, h)
    energy_QUBO = 2 * x_QUBO.T @ Q_from_SG @ x_QUBO + C_from_SG,
        with C_from_SG = 0.5 * np.sum(J) - np.sum(h)
    
    with sigma_Ising = 2*x_QUBO -1 
        
    """
    # off-diagonal terms
    Q = np.copy(J) # you MUST use np.copy to shallow copy it or you modify J!!
    # diagonal terms
    n = Q.shape[0]
    for j in range(n):
        Q[j,j] = h[j] - 1*np.sum(Q[:,j]) # no factor 2
    # constant
    C = 0.5 * np.sum(J) - np.sum(h)

    return Q, C


In [19]:
# Example Ising spin glass
N_verts = 5
N_edges = int(N_verts*(N_verts - 1)/2)

Jz, loc_fields, connect = generate_graph(N_verts, N_edges)
Jz_matrix = convert_to_Ising_matrix(N_verts, Jz)
### NB: you need to check if you want the 0.5 factor to be consistent with your TDVP implementation...
Jz_matrix_symmetrized = 0.5 * (Jz_matrix + Jz_matrix.T)

h_z_vector = loc_fields[:,1]

print("\n\n")
print(f"\n{Jz}")
print(f"\n{Jz_matrix}")
print(f"\n{Jz_matrix_symmetrized}")

print("\n\n")
print(f"\n{loc_fields}")
print(f"\n{h_z_vector}")

Q, C = Ising_to_QUBO(Jz_matrix_symmetrized, h_z_vector)

print("\n\n")
print(f"\n{Q}")
print(f"\n{C}")


### CHECK that Ising and QUBO are equivalent

# trial configs
M_trial = 3
trial_x = np.random.randint(0, 2, (M_trial, N_verts))
trial_sigma = 2*trial_x-1

# QUBO energy
energies_QUBO = 2*np.einsum("ki, ij, kj -> k", trial_x, Q, trial_x ) + C

# Ising energy
energies_Ising = 0.5*np.einsum("ki, ij, kj -> k", trial_sigma, Jz_matrix_symmetrized, trial_sigma ) + np.einsum("ki, i -> k", trial_sigma, h_z_vector )


#print(energies_QUBO==energies_Ising)

print("\n\n")
print(f"energies_QUBO = {energies_QUBO}")
print(f"energies_Ising = {energies_Ising}")


# so you can just solve the QUBO problem:  x.T @ Q @ x with the online QUBO solver
# the minimum energy for the corresponding spin glass problem is computed by using the formulas above for the CHECK






[[0.         1.         0.4240652 ]
 [0.         2.         0.80528274]
 [1.         2.         0.10013526]
 [0.         3.         0.88099915]
 [1.         3.         0.771633  ]
 [2.         3.         0.55125219]
 [0.         4.         0.33456745]
 [1.         4.         0.26419607]
 [2.         4.         0.41520782]
 [3.         4.         0.59421903]]

[[0.         0.4240652  0.80528274 0.88099915 0.33456745]
 [0.         0.         0.10013526 0.771633   0.26419607]
 [0.         0.         0.         0.55125219 0.41520782]
 [0.         0.         0.         0.         0.59421903]
 [0.         0.         0.         0.         0.        ]]

[[0.         0.2120326  0.40264137 0.44049958 0.16728372]
 [0.2120326  0.         0.05006763 0.3858165  0.13209804]
 [0.40264137 0.05006763 0.         0.27562609 0.20760391]
 [0.44049958 0.3858165  0.27562609 0.         0.29710951]
 [0.16728372 0.13209804 0.20760391 0.29710951 0.        ]]




[[ 0.         -0.39339307]
 [ 1.          0.489